# Code thu thập Data 

## Cài đặt các thư viện cần thiết

In [1]:
import requests, time, json
import pandas as pd
import networkx as nx
from itertools import combinations

## Cào Data từ OpenAlex sử dụng API 

In [2]:
BASE = "https://api.openalex.org/works"

def fetch_works_by_concept(concept_id="C2522767166", per_page=200, max_works=500, delay=0.5):
    works = []
    page = 1
    collected = 0
    while collected < max_works:
        params = {
            "filter": f"concepts.id:{concept_id}",
            "per_page": per_page,
            "page": page
        }
        resp = requests.get(BASE, params=params)
        if resp.status_code != 200:
            print("HTTP error", resp.status_code, resp.text)
            break
        data = resp.json()
        items = data.get("results", [])
        if not items:
            break
        works.extend(items)
        collected += len(items)
        page += 1
        time.sleep(delay)
        if len(items) < per_page:
            break
    return works

def works_to_excel(works, filename="works.xlsx"):
    rows = []
    for w in works:
        work_id = w.get("id")
        title = w.get("title")
        year = w.get("publication_year")
        cited = w.get("cited_by_count")
        
        # Tác giả (lấy cả tên và ID)
        authorships = w.get("authorships", [])
        authors = []
        author_ids = []
        for a in authorships:
            if "author" in a:
                authors.append(a["author"].get("display_name", ""))
                author_ids.append(a["author"].get("id", ""))
        
        authors_str = "; ".join(authors)
        author_ids_str = "; ".join(author_ids)
        
        # Concept (chỉ lấy top-3)
        concepts = w.get("concepts", [])
        top_concepts = [c["display_name"] for c in concepts[:3]]
        concepts_str = "; ".join(top_concepts)
        
        rows.append({
            "Work_ID": work_id,
            "Title": title,
            "Year": year,
            "Cited_by": cited,
            "Authors": authors_str,
            "Author_IDs": author_ids_str,
            "Concepts": concepts_str
        })
    
    df = pd.DataFrame(rows)
    df.to_excel(filename, index=False)
    print(f"Saved {len(df)} records to {filename}")

if __name__ == "__main__":
    works = fetch_works_by_concept("C2522767166", max_works=2400)
    works_to_excel(works, "Project_1_dataset.xlsx")


Saved 2400 records to Project_1_dataset.xlsx


Sau khi thu thập, ta thu được **2400 bài báo khoa học** của nhiều tác giả khác nhau được lưu vào trong Project_1_dataset.xlsx để tiện cho bước tiền xử lý tiếp theo. Bộ dữ liệu bao gồm các đặc trưng chính sau:

* **Work_ID**: Mã định danh duy nhất cho mỗi bài báo trong cơ sở dữ liệu OpenAlex.
* **Title**: Tiêu đề của bài báo.
* **Year**: Năm xuất bản bài báo.
* **Cited_by**: Số lần bài báo được trích dẫn, thể hiện mức độ ảnh hưởng của công trình trong cộng đồng khoa học.
* **Authors**: Danh sách tên tác giả của bài báo, phân tách bằng dấu “;”.
* **Author_IDs**: Mã định danh duy nhất cho từng tác giả trên OpenAlex, được dùng để phân biệt các tác giả trùng tên.
* **Concepts**: Các khái niệm hoặc lĩnh vực nghiên cứu chính của bài báo (lấy tối đa 3 concept nổi bật).

# Tiền xử lý dữ liệu

In [4]:
# Quan sát dữ liệu đã thu thập
df = pd.read_excel("Project_1_dataset.xlsx")
df.head()

,Work_ID,Title,Year,Cited_by,Authors,Author_IDs,Concepts
0,https://openalex.org/W2144634347,Molecular Cloning: A Laboratory Manual,2001,133517,Joseph Sambrook; Elisabeth Fritsch; Tom Maniatis,https://openalex.org/A5112152140; https://open...,Cloning (programming); Computational biology; ...
1,https://openalex.org/W4300870773,Statistical Power Analysis for the Behavioral ...,1989,83956,Keith E. Muller; Jacob Cohen,https://openalex.org/A5110163574; https://open...,Statistical power; Power analysis; Statistical...
2,https://openalex.org/W2156098321,Preferred reporting items for systematic revie...,2009,80944,David Moher; A. Liberati; Jennifer Tetzlaff; D...,https://openalex.org/A5068353058; https://open...,Systematic review; Statement (logic); Computer...
3,https://openalex.org/W2167279371,Fiji: an open-source platform for biological-i...,2012,61316,Johannes Schindelin; Ignacio Arganda‐Carreras;...,https://openalex.org/A5050085853; https://open...,Plug-in; Scripting language; Software
4,https://openalex.org/W3118615836,The PRISMA 2020 statement: an updated guidelin...,2021,58608,Matthew J. Page; Joanne E. McKenzie; Patrick M...,https://openalex.org/A5089979303; https://open...,Checklist; Systematic review; Guideline


5 mẫu dữ liệu đầu cho thấy các ID dùng để định danh bài báo và tác giả đang bị dính liền với địa chỉ web của OpenAlex. Bước đầu, ta tiến hành xử lý lại các ID này, việc tách riêng ID giúp dữ liệu gọn gàng hơn, thuận tiện cho việc xây dựng mạng lưới cộng tác và tránh nhầm lẫn khi phân tích

In [5]:
# Tiền xử lý 
df["Author_IDs"] = df["Author_IDs"].fillna("")
# Tách author_IDs khỏi link
df["Author_IDs"] = df["Author_IDs"].apply(
    lambda x: "; ".join([part.split("/")[-1].strip() for part in x.split(";")])
)
df = df[df["Author_IDs"].map(len) > 1]  # Chỉ giữ các bài có >1 tác giả
print(df["Author_IDs"].head())

#Tách Work_ID khỏi link
df["Work_ID"] = df["Work_ID"].apply(lambda x: x.split("/")[-1].strip())
print(df["Work_ID"].head())

0                A5112152140; A5047742196; A5091116725
1                             A5110163574; A5102808166
2    A5068353058; A5113580052; A5076572041; A504296...
3    A5050085853; A5021284069; A5005279299; A505575...
4    A5089979303; A5015844348; A5061726857; A506502...
Name: Author_IDs, dtype: object
0    W2144634347
1    W4300870773
2    W2156098321
3    W2167279371
4    W3118615836
Name: Work_ID, dtype: object


In [11]:
# Kiểm tra Work_ID có trùng không
print("Số Work_ID trùng:", df["Work_ID"].duplicated().sum())
# Kiểm tra có tác giả nào không có ID không
print("Số tác giả không có ID:", df["Author_IDs"].apply(lambda x: len(x) == 0).sum())
# Kiểm tra dư liệu có missing value không
print("Missing value:\n", df.isnull().sum())

Số Work_ID trùng: 0
Số tác giả không có ID: 0
Missing value:
 Work_ID       0
Title         6
Year          0
Cited_by      0
Authors       0
Author_IDs    0
Concepts      0
dtype: int64


Kết quả kiểm tra chất lượng dữ liệu cho thấy:

* **Số Work_ID trùng lặp:** 0 Không có bài báo nào bị trùng lặp.
* **Số tác giả không có Author_ID:** 0 toàn bộ tác giả đều được gán định danh rõ ràng.
* **Missing values:** chỉ có cột *Title* bị thiếu 6 giá trị, các cột khác không có giá trị khuyết.

Điều này chứng tỏ dữ liệu có độ sạch và độ tin cậy khá cao, thuận lợi cho các bước phân tích mạng lưới cộng tác và phát hiện cộng đồng.

Việc thiếu tiêu đề ở một số bài báo là không đáng lo ngại, vì các phân tích mạng lưới cộng tác chủ yếu dựa trên **Work_ID** và **Author_IDs** để xác định mối quan hệ giữa các tác giả. Do đó, ta có thể giữ nguyên dữ liệu mà không cần xử lý bổ sung cho cột *Title*.

In [13]:
# Lưu dư liệu đã làm sạch ra file CSV
output_file = "Project_1_dataset_clean.csv"
df.to_csv(output_file, index=False, encoding="utf-8-sig")

Sau bước tiền sử lý và kiểm tra chất lượng, ta lưu Dataset đa được làm sạch phục vụ cho bước tiếp theo

# Tạo đồ thị Mạng lưới cộng tác khoa học

In [14]:
# Đọc lại file CSV để kiểm tra
df = pd.read_csv("Project_1_dataset_clean.csv")
df.head()

,Work_ID,Title,Year,Cited_by,Authors,Author_IDs,Concepts
0,W2144634347,Molecular Cloning: A Laboratory Manual,2001,133517,Joseph Sambrook; Elisabeth Fritsch; Tom Maniatis,A5112152140; A5047742196; A5091116725,Cloning (programming); Computational biology; ...
1,W4300870773,Statistical Power Analysis for the Behavioral ...,1989,83956,Keith E. Muller; Jacob Cohen,A5110163574; A5102808166,Statistical power; Power analysis; Statistical...
2,W2156098321,Preferred reporting items for systematic revie...,2009,80944,David Moher; A. Liberati; Jennifer Tetzlaff; D...,A5068353058; A5113580052; A5076572041; A504296...,Systematic review; Statement (logic); Computer...
3,W2167279371,Fiji: an open-source platform for biological-i...,2012,61316,Johannes Schindelin; Ignacio Arganda‐Carreras;...,A5050085853; A5021284069; A5005279299; A505575...,Plug-in; Scripting language; Software
4,W3118615836,The PRISMA 2020 statement: an updated guidelin...,2021,58608,Matthew J. Page; Joanne E. McKenzie; Patrick M...,A5089979303; A5015844348; A5061726857; A506502...,Checklist; Systematic review; Guideline


In [15]:
import networkx as nx
from itertools import combinations


# Tạo graph
G = nx.Graph()

for _, row in df.iterrows():
    # Lấy danh sách tên tác giả
    authors = str(row["Authors"]).split(";")
    authors = [a.strip() for a in authors if a.strip()]
    
    # Lấy danh sách Author_IDs (đã được làm sạch, chỉ còn Axxxxx)
    author_ids = str(row["Author_IDs"]).split(";")
    author_ids = [a.strip() for a in author_ids if a.strip()]
    
    # Ghép ID và tên -> đảm bảo node là duy nhất theo ID
    author_nodes = []
    for aid, aname in zip(author_ids, authors):
        if aid:  
            node_id = aid  # sử dụng ID làm khóa chính
            G.add_node(node_id, name=aname)  # thêm thuộc tính 'name' để lưu tên hiển thị
            author_nodes.append(node_id)
    
    # Thêm cạnh đồng tác giả (nếu >=2 tác giả trong 1 bài)
    for a, b in combinations(author_nodes, 2):
        if G.has_edge(a, b):
            G[a][b]["weight"] += 1
        else:
            G.add_edge(a, b, weight=1)

# Kết quả
print("Số nodes (tác giả):", G.number_of_nodes())
print("Số edges (cộng tác):", G.number_of_edges())


Số nodes (tác giả): 9705
Số edges (cộng tác): 136267


Từ dữ liệu thu thập được, ta xây dựng được **mạng lưới cộng tác khoa học** giữa các tác giả:

* **Số nodes (tác giả):** 9,705
  → Mỗi node tương ứng với một tác giả duy nhất (được định danh bằng Author_ID).

* **Số edges (cộng tác):** 136,267
  → Mỗi cạnh biểu diễn mối quan hệ cộng tác (đồng tác giả trong ít nhất một bài báo). Trọng số (weight) của cạnh phản ánh số lần hai tác giả cùng xuất hiện trong các công trình.

Mạng lưới thu được có **quy mô lớn**, cho thấy sự kết nối rộng rãi giữa các nhà nghiên cứu trong lĩnh vực được chọn. Đây là cơ sở để áp dụng các thuật toán phân cụm (Louvain, Leiden, Fast Greedy) và các chỉ số đánh giá nhằm khám phá cấu trúc cộng đồng khoa học trong bộ dữ liệu.

In [19]:
# ===== Export nodes =====
nodes_data = []
for node, data in G.nodes(data=True):
    nodes_data.append({
        "Id": node,                 # Author_ID (duy nhất)
        "Label": data.get("name"),  # Tên tác giả
        "Degree": G.degree(node)    # Số cộng tác viên
    })
nodes_df = pd.DataFrame(nodes_data)
nodes_df.to_csv("Project_1_coauthor_nodes.csv", index=False, encoding="utf-8-sig")

# ===== Export edges =====
edges_data = []
for u, v, data in G.edges(data=True):
    edges_data.append({
        "Author1": G.nodes[u]["name"],   # Xuất bằng TÊN tác giả
        "Author2": G.nodes[v]["name"],
        "Weight": data["weight"]
    })
edges_df = pd.DataFrame(edges_data)
edges_df.to_csv("Project_1_coauthor_edges.csv", index=False, encoding="utf-8-sig")

print("Đã lưu nodes.csv và edges.csv thành công!")

Đã lưu nodes.csv và edges.csv thành công!


Ta lưu đồ thị đa tạo ra lại để phục vụ cho các bước tiếp theo